In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import time

In [2]:
gameRanks=pd.read_csv("GameRanks.csv")
gameIds= gameRanks['ID']

In [3]:
rootpath = 'https://boardgamegeek.com/xmlapi2/'
gamepath = 'thing/'
playpath = 'plays/'

In [37]:
gameparameters = {'id':gameIds[34],
              'stats' : 1}

playparameters = {"id": gameIds[34],
                  'type': 'thing',
                  'mindate':'2024-02-25'}

In [23]:
gamePlays=[]
for id in gameIds[0:300]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [ ]:
#took a break, now resuming
for id in gameIds[300:399]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [29]:
#Lost connection
#Restarting
for id in gameIds[399:]:
    playparameters = {"id": id, 
                      'type': 'thing',
                      'mindate':'2024-02-25'}
    r=requests.get(rootpath+playpath, params= playparameters)
    if r.status_code==429:
        time.sleep(30)
        r=requests.get(rootpath+playpath, params= playparameters)
        if r.status_code!=200: 
            print(f"Error Code:{r.status_code}, gameId {id}")
            break

    soup = BeautifulSoup(r.text, features='xml')
    gamePlays += re.findall('total="(\d+)"', str(soup.find('plays')))
    time.sleep(5)

In [35]:
gameRanks['Recorded Plays Last Month']=gamePlays

gameRanks=gameRanks[['ID', 'Recorded Plays Last Month', 'Overall Rank', 'Abstract Rank',
                     'Customizable Rank', 'Thematic Rank', 'Family Rank', 'Children Rank',
                     'Party Rank', 'Strategy Rank', 'War Rank']]

In [37]:
#Update CSV so I don't lose everything
#gameRanks.to_csv('GameRanks.csv')

In [47]:
##Now for Complexity, updated ranks, and wanting/wishing
gameparameters={"id": ",".join(map(str, gameIds)),
                'stats': 1}

r = requests.get(rootpath+gamepath, params=gameparameters)
r.status_code

200

In [48]:
soup = BeautifulSoup(r.text, features='xml')

In [54]:
names = re.findall('value="([^"]+)"', str(soup.find_all('name', {'type':'primary'})))

In [ ]:
gameRanks['Name']=names

In [64]:
ratings = re.findall('value="(\d+)"', str(soup.find_all('rank', {'name': 'boardgame'})))

In [79]:
gameRanks['Overall Rank']=ratings

In [106]:
complexity = re.findall('value="(\d\.*\d*)"', str(soup.find_all('averageweight')))
scores = re.findall('value="(\d\.*\d*)"', str(soup.find_all('bayesaverage')))
owned = re.findall('value="(\d+)"', str(soup.find_all('owned')))
wants = re.findall('value="(\d+)"', str(soup.find_all('wanting')))
wishes = re.findall('value="(\d+)"', str(soup.find_all('wishing')))

In [93]:
gameRanks['Complexity']=complexity
gameRanks['Rating']=scores
gameRanks['Owns']=owned
gameRanks['Wants']=wants
gameRanks['Wishlists']=wishes

In [118]:
gameRanks['Plays Last Month']=gameRanks['Recorded Plays Last Month']

In [119]:
gameRanks=gameRanks[['Name', 'ID', 'Rating', 'Complexity', 'Overall Rank', 'Plays Last Month',
                     'Owns', 'Wants', 'Wishlists', 'Abstract Rank', 'Customizable Rank', 'Thematic Rank',
                     'Family Rank', 'Children Rank', 'Party Rank', 'Strategy Rank', 'War Rank']]

In [120]:
#Write the Final CSV
#gameRanks.to_csv('GameRanks.csv')